# Projekt 6: Przewidywanie struktury drugorzędowej RNA

In [1]:
# %pip install tensorflow pandas tqdm argparse six

In [2]:
import os
from pathlib import Path

os.chdir("./SPOT-RNA")
DATASET_DIR = Path("../resources/datasets")

cores = max(1, os.cpu_count() - 1)

In [3]:
!python3 SPOT-RNA.py  --inputs {DATASET_DIR}/ArchiveII/5s_Acetobacter-aceti-2.fa  --outputs 'outputs/'  --cpu {cores}

>> Opening FASTA file...
>> Converting FASTA file from multiline to single line and writing to file.
>> Done!

Preparing tfr records file for SPOT-RNA:
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 31.69it/s]

Predicting for SPOT-RNA model 0
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.38it/s]

Predicting for SPOT-RNA model 1
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.04s/it]

Predicting for SPOT-RNA model 2
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]

Predicting for SPOT-RNA model 3
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.17s/it]

Predicting for SPOT-RNA model 4
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.18s/it]

Post Processing and Saving Output

Finished!

Processsing Time 11.704977035522461 seconds
